<a href="https://colab.research.google.com/github/thaismarques624/wie_gerador_certificado/blob/main/gerador.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


#GERADOR DE CERTIFICADOS


##Bibliotecas importantes

In [ ]:
!pip install PyPDF2
!pip install reportlab
from PyPDF2 import PdfReader, PdfWriter
from decimal import Decimal
from reportlab.pdfgen import canvas
from reportlab.lib.pagesizes import letter, A4
from reportlab.lib.units import inch
from google.colab import drive
import gspread
from google.colab import auth
from google.auth import default
import os
import io
import datetime

##Gerador do arquivo

In [ ]:
# Autenticar usando as credenciais padrão
auth.authenticate_user()

# Obter as credenciais
credentials, _ = default()

# Autorizar o acesso ao Google Sheets
gc = gspread.authorize(credentials)

def ler_informacoes_planilha(planilha_id, nome_aba):
    planilha = gc.open_by_key(planilha_id)
    aba = planilha.worksheet(nome_aba)
    dados = aba.get_all_values()
    return dados[0], dados[1:]

def ler_modelo_certificado(caminho_modelo_certificado):
    return PdfReader(caminho_modelo_certificado)

# Montar o Google Drive
drive.mount('/content/drive')


# Função para gerar certificado e salvar no arquivo de saída
def gerar_certificado(nome_aluno, tipo_evento, tipo_participante, local_evento, data_evento, duracao_evento, pdf_reader, caminho_arquivo_saida):
    pdf_writer = PdfWriter()

    for page_num in range(len(pdf_reader.pages)):
        page = pdf_reader.pages[page_num]
        packet = io.BytesIO()
        pagesize_per = (29.7 * inch, 21 * inch)
        c = canvas.Canvas(packet, pagesize = pagesize_per)
        c.setFont("Helvetica", 16)

        # Adicionar as informações do aluno no certificado apenas na primeira página
        if page_num == 0:
            # Adicionar as informações do aluno no certificado
            texto1 = f"Certifico que {nome_aluno}, participou do evento {tipo_evento} como {tipo_participante}, realizado no {local_evento} no dia {data_evento} com a duração de {duracao_evento}.\n"

            # Adicionar a data de impressão
            data_emissao = datetime.datetime.now().strftime("%d/%m/%Y")
            texto2 = f" Emitido em João Pessoa, {data_emissao}."

            texto = texto1 + "\n" + texto2
           # Definir a fonte e tamanho desejados

            font_size = 20.5

            # Calcular o tamanho do texto para ajustar o posicionamento
            texto_linhas = texto.split()
            text_height = font_size * len(texto_linhas) * 6  # Considerar espaçamento entre linhas

            # Definir a posição fixa logo abaixo do título
            y_start = 400  # Ajustar este valor conforme necessário para subir mais o texto

            # Calcular a largura máxima do texto para corresponder à largura disponível
            frame_x = 30  # Margem esquerda
            #frame_x_direita = 39  # Margem direita maior
            max_width = float(page.mediabox[3]) - frame_x * 1  # Subtrair margens esquerda e direita

            # Dividir o texto em linhas para caber na largura máxima
            texto_linhas = []
            linha_atual = ''
            for palavra in texto.split():
                if c.stringWidth(linha_atual + ' ' + palavra, "Helvetica", font_size) < max_width:
                    linha_atual += ' ' + palavra if linha_atual else palavra
                else:
                    texto_linhas.append(linha_atual)
                    linha_atual = palavra
            if linha_atual:
                texto_linhas.append(linha_atual)

            # Desenhar cada linha individualmente no canvas
            c.setFont("Helvetica", font_size)
            y = y_start
            for linha in texto_linhas:
                # Calcular a posição x para centralizar o texto horizontalmente
                line_width = c.stringWidth(linha, "Helvetica", font_size)
                x = (float(page.mediabox[2]) - line_width) / 2  # Centralizar horizontalmente
                c.drawString(x, y, linha)
                y -= font_size * 1.2  # Espaçamento entre linhas



        # Salvar o conteúdo do canvas
        c.save()

        # Adicionar a página ao escritor de PDF
        packet.seek(0)
        new_pdf = PdfReader(packet)
        page.merge_page(new_pdf.pages[0])
        pdf_writer.add_page(page)

        # Limpar o buffer
        packet.truncate(0)

    # Salvar o certificado no arquivo de saída
    with open(caminho_arquivo_saida, 'wb') as arquivo_saida:
        pdf_writer.write(arquivo_saida)

    return caminho_arquivo_saida

# ID da planilha do Google Sheets e nome da aba
planilha_id = 'https://docs.google.com/spreadsheets/d/coloque o id daqui/edit?gid=1697908971#gid=1697908971'
nome_aba = 'Coloque o nome da aba'

# Ler informações da planilha
cabecalho, linhas_planilha = ler_informacoes_planilha(planilha_id, nome_aba)
# Caminho para o modelo de certificado
caminho_modelo_certificado = 'copie o caminho do arquivo do seu drive'

# Ler o modelo de certificado
pdf_reader = ler_modelo_certificado(caminho_modelo_certificado)

# Função para salvar o arquivo no Google Drive
def salvar_no_drive(caminho_arquivo_saida, nome_arquivo_saida):
    caminho_completo = f'/copie o campinho de onde vai ficar salvo o arquivo/{nome_arquivo_saida}'

    # Verificar se o diretório de destino existe, se não, criar
    caminho_pasta = os.path.dirname(caminho_completo)
    if not os.path.exists(caminho_pasta):
        os.makedirs(caminho_pasta)

    with open(caminho_arquivo_saida, 'rb') as arquivo_entrada:
        with open(caminho_completo, 'wb') as arquivo_saida:
            arquivo_saida.write(arquivo_entrada.read())
    print(f"Certificado gerado e salvo em: {caminho_completo}")

# Iterar sobre cada linha da planilha
for linha in linhas_planilha:
    nome_aluno, tipo_evento, tipo_participante, local_evento, data_evento, duracao_evento = linha

    # Ler o modelo de certificado para cada linha
    pdf_reader = ler_modelo_certificado(caminho_modelo_certificado)

    # Gerar caminho único para o arquivo de certificado
    nome_arquivo_saida = f"certificado_{nome_aluno}.pdf"
    caminho_arquivo_saida = f'/content/{nome_arquivo_saida}'

    # Gerar e salvar o certificado para a linha atual
    caminho_certificado = gerar_certificado(nome_aluno, tipo_evento, tipo_participante, local_evento, data_evento, duracao_evento, pdf_reader, caminho_arquivo_saida)
    salvar_no_drive(caminho_certificado, nome_arquivo_saida)

# Finalizado
print("Processo de geração de certificados concluído.")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Certificado gerado e salvo em: /content/drive/My Drive/certificados_wie/Certificados gerados/certificado_Thais Gabrielly Marques de Andrade.pdf
Processo de geração de certificados concluído.


Referências:

In [ ]:
#https://www.reportlab.com/docs/reportlab-userguide.pdf
#https://stackoverflow.com/questions/5882636/how-to-create-a-pdf-document-with-differing-page-sizes-in-reportlab-python
